<a href="https://colab.research.google.com/github/vasu-rguktn/Team_X/blob/main/linear_regression_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!unzip archive.zip

Archive:  archive.zip
replace building_metadata.feather? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [17]:
!pip install pyarrow

In [24]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


train = pd.read_feather("train.feather")
weather = pd.read_feather("weather_train.feather")
building = pd.read_feather("building_metadata.feather")

print(train.shape, weather.shape, building.shape)


df = train.merge(building, on="building_id", how="left")
df = df.merge(weather, on=["site_id", "timestamp"], how="left")

df.head()


(20216100, 4) (139773, 9) (1449, 6)


,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
1,1,0,2016-01-01,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
2,2,0,2016-01-01,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
3,3,0,2016-01-01,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0
4,4,0,2016-01-01,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.700012,0.0,0.0


In [25]:
df["timestamp"] = pd.to_datetime(df["timestamp"])

df["hour"] = df["timestamp"].dt.hour
df["day"] = df["timestamp"].dt.day
df["month"] = df["timestamp"].dt.month
df["weekday"] = df["timestamp"].dt.weekday

df.fillna(df.median(numeric_only=True), inplace=True)



In [27]:
df = pd.get_dummies(df, columns=["primary_use"], drop_first=True)


In [30]:
df = df.sample(n=200000, random_state=42)

X = df.drop(columns=["meter_reading", "timestamp"])
y = df["meter_reading"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R² Score:", r2)


RMSE: 185007.46676344282
R² Score: 0.0021064971290043477


“The Linear Regression model performed poorly, with a very high RMSE and an R² score close to zero, indicating that it explains almost none of the variance in energy consumption. This shows that the relationship between building energy usage and the input features is highly nonlinear and influenced by complex interactions and temporal patterns that a simple linear model cannot capture. Therefore, Linear Regression serves only as a baseline, and more advanced nonlinear or ensemble models are required for accurate prediction.”

R² ≈ 0.002 → Model explains 0.2% of variance
→ Practically useless for prediction.

Huge RMSE → Prediction errors are very large relative to actual meter readings.